In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from prophet import Prophet
from prophet.plot import plot_plotly
import plotly.graph_objs as go

st.title('Stock Price Prediction App')

# Sidebar
st.sidebar.title('Settings')
stock_symbol = st.sidebar.text_input('Enter Stock Symbol (e.g., AAPL for Apple)', 'AAPL')
period = st.sidebar.selectbox('Select Time Period', ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'])

def load_data(symbol, period):
    data = yf.download(symbol, period=period)
    return data

data_load_state = st.text('Loading data...')
data = load_data(stock_symbol, period)
data_load_state.text('Data loaded successfully!')

st.subheader('Raw Data')
st.write(data)

# Plotting
st.subheader('Interactive Stock Price Plot')
trace1 = go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close')
layout = go.Layout(title='Interactive Stock Price Plot', xaxis_title='Date', yaxis_title='Close Price')
fig = go.Figure(data=[trace1], layout=layout)
st.plotly_chart(fig, use_container_width=True)

# LSTM Model
st.subheader('LSTM Model')

# Prepare data for LSTM
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Close']])

window_size = 60
X, y = [], []
for i in range(window_size, len(data)):
    X.append(scaled_data[i-window_size:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)

X = X.reshape((X.shape[0], X.shape[1], 1))

# Build LSTM model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)),
    LSTM(units=50, return_sequences=False),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(X[-1].reshape(1, window_size, 1))
predicted_price = scaler.inverse_transform(predictions)[0][0]

st.write(f'Predicted Closing Price (Next Day): {predicted_price}')

# Prophet Forecasting
st.subheader('Prophet Forecasting')

df_prophet = data.reset_index()[['Date', 'Close']].rename(columns={'Date':'ds', 'Close':'y'})
m = Prophet()
m.fit(df_prophet)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
fig = plot_plotly(m, forecast)
st.plotly_chart(fig)


2024-06-15 20:40:39.594 
  command:

    streamlit run C:\Users\91885\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
[*********************100%%**********************]  1 of 1 completed


IndexError: tuple index out of range